# Dataclasses
- Author: 楊晴雯
- Date: 2023/05/25 
- new since Python 3.7 
- use @dataclass decorator (`from dataclasses import dataclass`)

## Reference
- [Real Python](https://realpython.com/python-data-classes/)

## Vanilla Data Class
- Card Class 撲克牌
- Regular Class 印出來會是一個物件，not very descriptive，
可以使用 `__repr__` 來換成你想要的表示方式。Data Class 印出來則非常明確。


In [1]:
# Card Class
from dataclasses import dataclass 

@dataclass 
class Card:
    rank: str 
    suit: str 
    
spadesA= Card('A', 'spades')
heartA= Card('A', 'heart')
heartA

Card(rank='A', suit='heart')

In [2]:
# Alterniative: Create dataclass 
from dataclasses import make_dataclass 
Card = make_dataclass('Card', ['rank', 'suit'])
spadesA= Card('A', 'spades')
spadesA 

Card(rank='A', suit='spades')

- default values 
- type hint 
## Note 
- 2 dataclass objects are equal if all of their fields are equal!!

## Radian
-![](images/radian_explain.png)
-![](images/haversine_formula.png)

## Data class with class method 
- Position 

In [3]:
# Default Values
from math import radians, cos, sin, asin, sqrt # asin: arcsin 
# radian 弧度： 1 radian = 180/π = 57.3° 

@dataclass 
class Position:
    name: str 
    lon: float = 0.0
    lat: float = 0.0

    # method in dataclass 
    def distance_to(self, other):
        r = 6371 # Earth radius in kilometers 
        lon1, lon2 = radians(self.lon), radians(other.lon)
        lat1, lat2 = radians(self.lat), radians(other.lat) 
        # haversine formula: measure distance between two points on a sphere 
        h = (sin((lat2-lat1)/2)**2 + cos(lat1)*cos(lat2)*sin((lon2-lon1)/2)**2)
        return 2*r*asin(sqrt(h))



Island1 = Position('Island1', 23.5, 45.6)
Island2 = Position('Island1', 23.5, 45.6)

# Taiwan Hsinchu 
Hsinchu = Position('Hsinchu', 24.47, 120.97)
MtFuji = Position('MtFuji', 35.36, 138.73) 

# check equality
print(Island1 == Island2)
# check distance
Island1.distance_to(Island2)

# check Hsinchu and Mt.Fuji distance
print('Distance from Hsinchu to Mt.Fuji:', Hsinchu.distance_to(MtFuji), 'km') # km 
# check Hsinchu and Mt.Fuji distance reversely 
MtFuji.distance_to(Hsinchu)  == Hsinchu.distance_to(MtFuji)


True
Distance from Hsinchu to Mt.Fuji: 2115.462497170761 km


True

## Use another class to manipulate a data class 
- Use `Deck` to manipulate `Card`
- Beware of how you initialize the deck of cards (`default_factory` argument)

In [4]:
from typing import List
from dataclasses import dataclass, field


all_suits = '♠ ♡ ♢ ♣'.split()
all_ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()



Don’t do this! This introduces one of the most common anti-patterns in Python: using mutable default 
arguments. The problem is that all instances of Deck will use the same list object as the default 
value of the .cards property. This means that if, say, one card is removed from one Deck, 
then it disappears from all other instances of Deck as well. Actually, data classes try to prevent you 


from doing this, and the code above will raise a ValueError.



In [5]:
def make_french_deck():
    return [Card(r, s) for s in all_suits for r in all_ranks] 
try:
    @dataclass    
    class Deck:
        cards: List[Card] = make_french_deck()
except ValueError as e:
    print(e)
# ValueError: mutable default  for field cards is not allowed: use default_factory

mutable default <class 'list'> for field cards is not allowed: use default_factory


### ⛔️ However, note that removing dataclass decorator will not raise error


In [6]:
class Deck:
    cards: List[Card] = make_french_deck()

deck1 = Deck()
deck2 = Deck()
# remove spades A from deck1
deck1.cards 
# check spades A is still in deck2
deck2.cards[-1] is spadesA 
print(deck2.cards[:10])

[Card(rank='2', suit='♠'), Card(rank='3', suit='♠'), Card(rank='4', suit='♠'), Card(rank='5', suit='♠'), Card(rank='6', suit='♠'), Card(rank='7', suit='♠'), Card(rank='8', suit='♠'), Card(rank='9', suit='♠'), Card(rank='10', suit='♠'), Card(rank='J', suit='♠')]


In [22]:
@dataclass
class Deck:
    cards: List[Card] = field(default_factory=make_french_deck) 

## field

`field()` 是拿來包裝 `Field` class 免得讓他直接暴露。
This function is used instead of exposing Field creation directly,
so that a type checker can be told (via overloads) that this is a
function whose type depends on its parameters.

- default: Default value of the field
- default_factory: Function that returns the initial value of the field
- init: Use field in `.__init__()` method? (Default is True.)
- repr: Use field in repr of the object? (Default is True.)
- compare: Include the field in comparisons? (Default is True.)
- hash: Include the field when calculating hash()? (Default is to use the same as for compare.)
- metadata: A mapping with information about the field


```python 
class Field:
    __slots__ = ('name',
                 'type',
                 'default',
                 'default_factory',
                 'repr',
                 'hash',
                 'init',
                 'compare',
                 'metadata',
                 '_field_type',  # Private: not to be used by user code.
                 )

    def __init__(self, default, default_factory, init, repr, hash, compare,
                 metadata):
        self.name = None
        self.type = None
        self.default = default
        self.default_factory = default_factory
        self.init = init
        self.repr = repr
        self.hash = hash
        self.compare = compare
        self.metadata = (_EMPTY_METADATA
                         if metadata is None else
                         types.MappingProxyType(metadata))
        self._field_type = None

    def __repr__(self):
        return ('Field('
                f'name={self.name!r},'
                f'type={self.type!r},'
                f'default={self.default!r},'
                f'default_factory={self.default_factory!r},'
                f'init={self.init!r},'
                f'repr={self.repr!r},'
                f'hash={self.hash!r},'
                f'compare={self.compare!r},'
                f'metadata={self.metadata!r},'
                f'_field_type={self._field_type}'
                ')')

    # This is used to support the PEP 487 __set_name__ protocol in the
    # case where we're using a field that contains a descriptor as a
    # default value.  For details on __set_name__, see
    # https://www.python.org/dev/peps/pep-0487/#implementation-details.
    #
    # Note that in _process_class, this Field object is overwritten
    # with the default value, so the end result is a descriptor that
    # had __set_name__ called on it at the right time.
    def __set_name__(self, owner, name):
        func = getattr(type(self.default), '__set_name__', None)
        if func:
            func(self.default, owner, name)
```

In [8]:
from dataclasses import fields
fields(Card)

(Field(name='rank',type='typing.Any',default=<dataclasses._MISSING_TYPE object at 0x7f2617e6dfa0>,default_factory=<dataclasses._MISSING_TYPE object at 0x7f2617e6dfa0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 Field(name='suit',type='typing.Any',default=<dataclasses._MISSING_TYPE object at 0x7f2617e6dfa0>,default_factory=<dataclasses._MISSING_TYPE object at 0x7f2617e6dfa0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD))

In [14]:
# however, for Deck, it is too verbose 
print(fields(Deck)) 
# because it lists all of the cards
# we can manually create a less verbose 

(Field(name='cards',type=typing.List[types.Card],default=<dataclasses._MISSING_TYPE object at 0x7f2617e6dfa0>,default_factory=[Card(rank='2', suit='♠'), Card(rank='3', suit='♠'), Card(rank='4', suit='♠'), Card(rank='5', suit='♠'), Card(rank='6', suit='♠'), Card(rank='7', suit='♠'), Card(rank='8', suit='♠'), Card(rank='9', suit='♠'), Card(rank='10', suit='♠'), Card(rank='J', suit='♠'), Card(rank='Q', suit='♠'), Card(rank='K', suit='♠'), Card(rank='A', suit='♠'), Card(rank='2', suit='♡'), Card(rank='3', suit='♡'), Card(rank='4', suit='♡'), Card(rank='5', suit='♡'), Card(rank='6', suit='♡'), Card(rank='7', suit='♡'), Card(rank='8', suit='♡'), Card(rank='9', suit='♡'), Card(rank='10', suit='♡'), Card(rank='J', suit='♡'), Card(rank='Q', suit='♡'), Card(rank='K', suit='♡'), Card(rank='A', suit='♡'), Card(rank='2', suit='♢'), Card(rank='3', suit='♢'), Card(rank='4', suit='♢'), Card(rank='5', suit='♢'), Card(rank='6', suit='♢'), Card(rank='7', suit='♢'), Card(rank='8', suit='♢'), Card(rank='9'

In [26]:


def make_french_deck():
    return [Card(r, s) for s in all_suits for r in all_ranks] 
@dataclass 
class Card:
    rank: str 
    suit: str 

@dataclass
class Deck:
    cards: List[Card] = field(default_factory=make_french_deck)
    def __repr__(self):
        cards = ', '.join(f'{c.suit}{c.rank}' for c in self.cards)
        # self.__class__.__name__ is the current class name
        # namely, Deck 
        return f'{self.__class__.__name__}({cards})'
Deck()

Deck(♠2, ♠3, ♠4, ♠5, ♠6, ♠7, ♠8, ♠9, ♠10, ♠J, ♠Q, ♠K, ♠A, ♡2, ♡3, ♡4, ♡5, ♡6, ♡7, ♡8, ♡9, ♡10, ♡J, ♡Q, ♡K, ♡A, ♢2, ♢3, ♢4, ♢5, ♢6, ♢7, ♢8, ♢9, ♢10, ♢J, ♢Q, ♢K, ♢A, ♣2, ♣3, ♣4, ♣5, ♣6, ♣7, ♣8, ♣9, ♣10, ♣J, ♣Q, ♣K, ♣A)

## Comparing Cards
- eq.
- sort

In [32]:
# eq. 
# order
from dataclasses import dataclass, field


@dataclass(order=True)
class Card:
    sort_index: int = field(init=False, repr=False)
    rank: str
    suit: str

    def __post_init__(self):
        self.sort_index = (all_ranks.index(self.rank) * len(all_suits)
                           + all_suits.index(self.suit))

    def __str__(self):
        return f'{self.suit}{self.rank}'

In [24]:
queen_of_hearts = Card('Q', '♡')
ace_of_spades = Card('A', '♠')
ace_of_spades > queen_of_hearts

True

In [35]:
from random import sample
# order = True 

ksamples = Deck(sample(make_french_deck(), k=10))
# how to sort the cards?    
ksamples.cards.sort()

In [36]:
ksamples

Deck(♠2, ♣2, ♣3, ♡6, ♡8, ♣8, ♠10, ♣10, ♡K, ♣K)

## Inheritance 

In [ ]:
from dataclasses import dataclass

@dataclass
class Position:
    name: str
    lon: float
    lat: float

@dataclass
class Capital(Position):
    country: str